<a href="https://colab.research.google.com/github/prateekbeh/algo_ds/blob/master/seismic_lwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset as dt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.dates as mdt
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
from datetime import datetime
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.animation as anim
from shapely.geometry import Point , shape , Polygon
from dataclasses import dataclass, fields
import matplotlib.pyplot as pl
import hvplot.xarray
#import fiona
import shapefile
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import netCDF4 as nc
import math
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from datetime import date
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 70.0 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
!pip install netCDF4


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']
catalog_data = catalog_data[catalog_data['mag'] >= 3]

# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.5
catalog_data['lat_grid'] = np.where((catalog_data['latitude'] % 1) < 0.5,
                                    np.floor(catalog_data['latitude']) + 0.25,
                                    np.floor(catalog_data['latitude']) + 0.75)

catalog_data['lon_grid'] = np.where((catalog_data['longitude'] % 1) < 0.5,
                                     np.floor(catalog_data['longitude']) + 0.25,
                                     np.floor(catalog_data['longitude']) + 0.75)



# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
print("Number of unique grid points for each month:")
print(unique_grid_points_per_month)


# Display the result
events_per_month.head()

# New Step 7: Read a NetCDF file (GRACE Mascon data)
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]

    # Convert longitudes from [0, 360] to [-180, 180] range
    longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 31, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)

# Ensure correct column names for grace_data
# No need to rename columns, keep as 'latitude' and 'longitude'

# Step 9: Display columns for debugging
print("GRACE Data Columns:")
print(grace_data.columns.tolist())
print("Events per Month Columns:")
print(events_per_month.columns.tolist())
print(events_per_month.columns)

# Step 9: Based on the printed column names, adjust the merge operation
grace_filtered_data = grace_data.merge(events_per_month,
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-3-336d50d7b15a>:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


Number of unique grid points for each month:
    year_month  unique_lat_grid_count  unique_lon_grid_count
0      2002-04                      7                      8
1      2002-05                     12                     10
2      2002-06                      8                      9
3      2002-07                     10                     12
4      2002-08                      9                     10
..         ...                    ...                    ...
256    2023-08                     12                     14
257    2023-09                     10                     11
258    2023-10                     12                     12
259    2023-11                     10                     10
260    2023-12                     10                      7

[261 rows x 3 columns]
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Variables in GRACE Mascon NetCDF file:
dict_keys(['lon', 'lat', 'time'

In [ ]:
grace_filtered_data

,year_month,latitude,longitude,lwe_thickness,lat_grid,lon_grid,event_count,lat_list,lon_list,max_mag,min_mag,avg_mag
0,2002-04,31.75,-116.25,-0.007192,31.75,-116.25,1,[31.8211667],[-116.2233333],3.16,3.16,3.160000
1,2002-04,32.25,-115.25,2.824223,32.25,-115.25,3,"[32.1051667, 32.3441667, 32.2871667]","[-115.4556667, -115.313, -115.3163333]",3.30,3.01,3.203333
2,2002-04,32.25,-114.75,2.007574,32.25,-114.75,1,[32.0535],[-114.9573333],4.10,4.10,4.100000
3,2002-04,34.75,-116.25,5.708782,34.75,-116.25,1,[34.524],[-116.2946667],4.39,4.39,4.390000
4,2002-04,36.75,-121.75,4.652653,36.75,-121.75,2,"[36.8671667, 36.9605]","[-121.607, -121.5906667]",3.60,3.11,3.355000
...,...,...,...,...,...,...,...,...,...,...,...,...
3684,2023-12,35.75,-117.75,-37.412506,35.75,-117.75,3,"[35.9288333, 35.9253333, 35.7473333]","[-117.6818333, -117.684, -117.5703333]",3.42,3.18,3.306667
3685,2023-12,36.75,-120.75,-39.046023,36.75,-120.75,1,[36.8508333],[-120.9696667],3.70,3.70,3.700000
3686,2023-12,37.25,-118.75,-36.929195,37.25,-118.75,1,[37.0655],[-118.5315],3.49,3.49,3.490000
3687,2023-12,38.75,-122.75,-20.952386,38.75,-122.75,2,"[38.7968333, 38.814]","[-122.746, -122.7988333]",3.50,3.41,3.455000


In [ ]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [ ]:
!pip install keras_tuner
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.LSTM(n_neurons, return_sequences=True, input_shape=[1, 3] , activation = "tanh"))
    for _ in range(n_hidden):
        return_sequences = True if _ < n_hidden - 1 else False
        model.add(tf.keras.layers.LSTM(n_neurons, return_sequences=return_sequences, activation="tanh"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss="mse", optimizer=optimizer, # Changed loss to 'mse' for regression
                  metrics=["mae"]) # Changed metric to 'mae' for regression
    return model

    """
    tf.keras.layers.LSTM(n_neurons, return_sequences=True, input_shape=[None, 3] , activation = "tanh"),
    for _ in range(n_hidden):
        model.add(tf.keras.layers.LSTM(n_neurons, return_sequences=True, activation="tanh"))
    model.add(tf.keras.layers.Dense(1));
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model
    """

In [ ]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs,
                        callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae

In [ ]:
# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness', 'event_count', 'max_mag', 'min_mag', 'avg_mag']]


# Step 10: Prepare data for LSTM model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Extract predictors and target
X = grace_filtered_df[['event_count', 'year_month','max_mag', 'min_mag', 'avg_mag','lwe_thickness']]
y = grace_filtered_df[['year_month','lwe_thickness']]

X = X.set_index('year_month')

X_train = X.loc['2002-04':'2013-05', ['event_count', 'max_mag', 'avg_mag']]
X_valid = X.loc['2013-06':'2020-05', ['event_count', 'max_mag', 'avg_mag']]
X_test = X.loc['2020-06':'2024-01', ['event_count', 'max_mag', 'avg_mag']]

y_train = X.loc['2002-04':'2013-05', 'lwe_thickness']
y_valid = X.loc['2013-06':'2020-05', 'lwe_thickness']

tf.random.set_seed(42)
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    X_train.to_numpy(),
    targets=y_train,
    sequence_length=1,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    X_valid.to_numpy(),
    targets=y_valid,
    sequence_length=1,
    batch_size=32
)

random_search_tuner = kt.RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=10,
    overwrite=True,
    directory="/content/drive/MyDrive/grace_data/tuner_srch",
    project_name="hp_search"
)

rst = random_search_tuner.search(train_ds, epochs=10, validation_data=valid_ds)

best_hp = random_search_tuner.get_best_hyperparameters()[0]
model = random_search_tuner.hypermodel.build(best_hp)
"""
deep_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 3]),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1)
])
"""

fit_and_evaluate(model, train_ds , valid_ds , 1e-4 )

#,'max_mag', 'min_mag', 'avg_mag','lwe_thickness'

Trial 10 Complete [00h 01m 14s]
val_mae: 15.357110023498535

Best val_mae So Far: 15.353927612304688
Total elapsed time: 00h 07m 46s
Epoch 1/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 65ms/step - loss: 5.4037 - mae: 5.8821 - val_loss: 15.3124 - val_mae: 15.8079
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 5.2836 - mae: 5.7619 - val_loss: 15.3053 - val_mae: 15.8008
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 5.3830 - mae: 5.8611 - val_loss: 15.2982 - val_mae: 15.7937
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 5.3225 - mae: 5.8005 - val_loss: 15.2916 - val_mae: 15.7871
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 5.3232 - mae: 5.8022 - val_loss: 15.2849 - val_mae: 15.7804
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - loss: 5.3085 - mae: 5.7877 - val_loss: 15.2782 - val_mae: 15.7738
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 5.3647 - mae: 5.8436 - val_loss: 15.2716 - val_mae: 15.7672
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━

15.541095733642578

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 1, 250)              │         254,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 1, 250)              │         501,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 1, 250)              │         501,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 1, 250)              │         501,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 250)                 │         501,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             251 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,516,504 (17.23 MB)

 Trainable params: 2,258,251 (8.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,258,253 (8.61 MB)

In [ ]:
X_test_reshaped = X_test.to_numpy().reshape(X_test.shape[0], 1, X_test.shape[1])


In [ ]:
X_test

,event_count,max_mag,avg_mag
year_month,,,
2020-06,1,3.21,3.210000
2020-06,2,3.26,3.145000
2020-06,1,3.70,3.700000
2020-06,1,3.00,3.000000
2020-06,1,3.18,3.180000
...,...,...,...
2023-12,3,3.42,3.306667
2023-12,1,3.70,3.700000
2023-12,1,3.49,3.490000


In [ ]:
best_hp.values

{'n_hidden': 4,
 'n_neurons': 250,
 'learning_rate': 0.00020874518249741858,
 'optimizer': 'adam'}

In [ ]:
# prompt: convert X_test_reshaped to a dataframe with first col as   event_count , second col as max_mag and third col as avg_mag. show first 10 rows

import pandas as pd

# Assuming X_test_reshaped is already defined as in your provided code
# X_test_reshaped = X_test.to_numpy().reshape(X_test.shape[0], 1, X_test.shape[1])

# Create a DataFrame from X_test_reshaped
X_test_df = pd.DataFrame(X_test_reshaped.reshape(X_test.shape[0], X_test.shape[1]), columns=['event_count', 'max_mag', 'avg_mag'])

# Display the first 10 rows
print(X_test_df.head(10))

   event_count  max_mag   avg_mag
0          1.0     3.21  3.210000
1          2.0     3.26  3.145000
2          1.0     3.70  3.700000
3          1.0     3.00  3.000000
4          1.0     3.18  3.180000
5          1.0     3.90  3.900000
6          2.0     3.62  3.380000
7          9.0     5.51  3.650000
8         28.0     5.80  3.597143
9          1.0     3.16  3.160000


In [ ]:
# prompt: make a df from model.predict(X_test_reshaped) and name the col as lwe. show first 10 rows

# Assuming 'model' and 'X_test_reshaped' are defined as in your previous code.
import pandas as pd

predictions = model.predict(X_test_reshaped)
# Create a DataFrame from the predictions
df_predictions = pd.DataFrame(predictions, columns=['lwe'])
# Show the first 10 rows
print(df_predictions.head(10))

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
        lwe
0 -0.362477
1 -0.362483
2 -0.362482
3 -0.362476
4 -0.362477
5 -0.362484
6 -0.362485
7 -0.362511
8 -0.362549
9 -0.362477


In [ ]:
model.predict(X_test_reshaped)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[-0.36247748],
       [-0.36248347],
       [-0.36248216],
       [-0.36247554],
       [-0.36247718],
       [-0.3624842 ],
       [-0.36248472],
       [-0.36251143],
       [-0.3625492 ],
       [-0.362477  ],
       [-0.36247802],
       [-0.36247888],
       [-0.36247736],
       [-0.36248586],
       [-0.36254197],
       [-0.36258885],
       [-0.36247766],
       [-0.36248317],
       [-0.36248624],
       [-0.36247945],
       [-0.36248344],
       [-0.3624922 ],
       [-0.36248118],
       [-0.3624758 ],
       [-0.3624839 ],
       [-0.36249533],
       [-0.3624758 ],
       [-0.36247832],
       [-0.36247945],
       [-0.36251318],
       [-0.3624927 ],
       [-0.36251098],
       [-0.36254257],
       [-0.3624849 ],
       [-0.36247635],
       [-0.36251748],
       [-0.3624926 ],
       [-0.36248168],
       [-0.3625424 ],
       [-0.36248022],
       [-0.362477  ],
       [-0.36248463],
       [-0.36247665],
       [-0.36247563],
       [-0.36247975],
       [-0